## Agente Planejador-Executor

In [3]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model_name="gemini-1.5-flash")

llm.invoke("Olá, Mundo!").content

'Olá, Mundo! 👋\n\nQue bom te ver por aqui! 😄 \n\nO que você gostaria de fazer hoje? 🤔 \n'

In [7]:
# Definindo o estado

from typing_extensions import TypedDict
from typing import List, Annotated
import operator

class StatePlan(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[tuple], operator.add]
    response: str

In [8]:
# Estrutura de resposta do LLM para a etapa de planejamento

from pydantic import BaseModel, Field

class Plan(BaseModel):
    """Planeje seguir no futuro"""
    plan: List[str] = Field(
        description="Lista de etapas para seguir, devem estar classificadas em ordem de execução"
    )

In [17]:
# Nó de planejamento

from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Planeje as atividades que precisam ser seguidas para responder a pergunta do usuário"
        ),
        (
            "placeholder",
            "{messages}"
        ),
    ]
)

llm_estruturado = llm.with_structured_output(Plan)
planner = planner_prompt | llm_estruturado

In [19]:
# Testando o planner

planner.invoke(
    {
        "messages": [
            ("user", "Quais os apartamentos disponíveis para locação em Fortaleza? ")
        ]
    }
)

Plan(plan=['Obter dados de imóveis para alugar em Fortaleza', 'Filtrar os dados para encontrar apenas apartamentos', 'Retornar a lista de apartamentos disponíveis'])